In [1]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources for Arabic
nltk.download('arabic')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import ngrams

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
import spacy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Error loading arabic: Package 'arabic' not found in index
/kaggle/input/arabic-levantine-hate-speech-detection/train.csv
/kaggle/input/arabic-levantine-hate-speech-detection/test.csv
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.tokenize import RegexpTokenizer

def arabic_tokenizer(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    return tokens

In [4]:
dfTrain=pd.read_csv("/kaggle/input/arabic-levantine-hate-speech-detection/train.csv")
dfTest=pd.read_csv("/kaggle/input/arabic-levantine-hate-speech-detection/test.csv")

In [5]:
dfTrainN=dfTrain
dfTestN=dfTest

In [6]:
dfTrainN["Class_new"]=1
dfTrainN.loc[dfTrainN["Class"] == "normal", "Class_new"] = 0

In [7]:
dfTestN["Class_new"]=1
dfTestN.loc[dfTestN["Class"] == "normal", "Class_new"] = 0

In [8]:
#dfTrainN['Tweet'] = dfTrainN['Tweet'].apply(lambda x: ' '.join(x))

In [9]:
dfTrainN['Tweet'] = dfTrainN['Tweet'].apply(lambda x: list(ngrams(x.split(), 1)))

In [10]:
dfTestN['Tweet'] = dfTestN['Tweet'].apply(lambda x: list(ngrams(x.split(), 1)))

In [11]:
dfTrainN

,Tweet,Class,Class_new
0,"[(وزير,), (الخارجية,), (اللبناني,), (جبران,), ...",normal,0
1,"[(سورية,), (بلد,), (الحضارات,), (تربطها,), (بع...",normal,0
2,"[(اخي,), (الحاج,), (اذا,), (شعرت,), (انك,), (م...",normal,0
3,"[(ما,), (فيك,), (تعيش,), (بلا,), (ما,), (تكب,)...",normal,0
4,"[(هذا,), (البطل,), (الذي,), (قاتل,), (وجاذف,),...",abusive,1
...,...,...,...
4671,"[(كول,), (هوا,), (مرة,), (تانيي,), (وحلوا,), (...",abusive,1
4672,"[(رئيس,), (روحي؟,), (تروح,), (روحك,), (انت,), ...",abusive,1
4673,"[(إذا,), (أرادت,), (إسرائيل,), (أن,), (تضمن,),...",normal,0
4674,"[(خليك,), (بحالك,), (يا,), (نعيمي,), (على,), (...",abusive,1


In [12]:
dfTestN

,Tweet,Class,Class_new
0,"[(وحاج,), (حكي,), (مثاليات,), (وامه,), (ومابعر...",normal,0
1,"[(قطر,), (مرادف,), (الارهاب,)]",hate,1
2,"[(ارمني,), (يوناني,), (يحمل,), (جنسية,), (الما...",hate,1
3,"[(يا,), (حمار,), (قمة,), (يعني,), (زبالة,)]",abusive,1
4,"[(ما,), (حدا,), (غير,), (يلي,), (خلفتك,), (عاه...",abusive,1
...,...,...,...
1165,"[(ههههههه,), (على,), (فكرة,), (بس,), (اقرأي,),...",abusive,1
1166,"[(طول,), (ما,), (جبران,), (باسيل,), (ورا,), (م...",normal,0
1167,"[(يلعن,), (روحه,), (من,), (الجد,), (للولد,), (...",abusive,1
1168,"[(هاي,), (يا,), (غلن؟إي,), (إنت,), (أول,), (شي...",abusive,1


In [13]:
dfTrainN=dfTrainN.explode('Tweet')

In [14]:
dfTestN=dfTestN.explode('Tweet')

In [15]:
dfTrainN

,Tweet,Class,Class_new
0,"(وزير,)",normal,0
0,"(الخارجية,)",normal,0
0,"(اللبناني,)",normal,0
0,"(جبران,)",normal,0
0,"(باسيل,)",normal,0
...,...,...,...
4675,"(عصام,)",normal,0
4675,"(نعيم,)",normal,0
4675,"(جبران,)",normal,0
4675,"(باسيل,)",normal,0


In [16]:
dfTestN

,Tweet,Class,Class_new
0,"(وحاج,)",normal,0
0,"(حكي,)",normal,0
0,"(مثاليات,)",normal,0
0,"(وامه,)",normal,0
0,"(ومابعرف,)",normal,0
...,...,...,...
1169,"(نفخ,)",normal,0
1169,"(شفط,)",normal,0
1169,"(او,)",normal,0
1169,"(بس,)",normal,0


In [17]:
stop_words = set(stopwords.words('arabic'))

In [18]:
dfTrainN = dfTrainN[~dfTrainN['Tweet'].apply(lambda x: any(word in stop_words for word in x))]

In [19]:
dfTestN = dfTestN[~dfTestN['Tweet'].apply(lambda x: any(word in stop_words for word in x))]

In [20]:
dfTrainN

,Tweet,Class,Class_new
0,"(وزير,)",normal,0
0,"(الخارجية,)",normal,0
0,"(اللبناني,)",normal,0
0,"(جبران,)",normal,0
0,"(باسيل,)",normal,0
...,...,...,...
4675,"(عصام,)",normal,0
4675,"(نعيم,)",normal,0
4675,"(جبران,)",normal,0
4675,"(باسيل,)",normal,0


In [21]:
dfTestN

,Tweet,Class,Class_new
0,"(وحاج,)",normal,0
0,"(حكي,)",normal,0
0,"(مثاليات,)",normal,0
0,"(وامه,)",normal,0
0,"(ومابعرف,)",normal,0
...,...,...,...
1169,"(تكبير,)",normal,0
1169,"(نفخ,)",normal,0
1169,"(شفط,)",normal,0
1169,"(او,)",normal,0


In [22]:
dfTrainN['Tweet'].value_counts()

(جبران,)     866
(باسيل,)     853
(انت,)       416
(الله,)      287
(شو,)        285
            ... 
(عنوانك,)      1
(الواضح,)      1
(يامير,)       1
(مصمم,)        1
(فشخكم,)       1
Name: Tweet, Length: 18043, dtype: int64

In [23]:
 dfTrainN.groupby('Class_new')["Tweet"].apply(lambda x: x.value_counts())

Class_new             
0          (جبران,)       751
           (باسيل,)       742
           (عم,)          182
           (الله,)        175
           (لبنان,)       174
                         ... 
1          (بقصد,)          1
           (عامليتلك,)      1
           (الزرقا,)        1
           (الخرزة,)        1
           (فشخكم,)         1
Name: Tweet, Length: 20732, dtype: int64

In [24]:
feature = dfTrainN.Tweet
target = dfTrainN.Class_new
feature1 = dfTestN.Tweet
target1 = dfTestN.Class_new

In [25]:
feature

0           (وزير,)
0       (الخارجية,)
0       (اللبناني,)
0          (جبران,)
0          (باسيل,)
           ...     
4675        (عصام,)
4675        (نعيم,)
4675       (جبران,)
4675       (باسيل,)
4675      (والنعم,)
Name: Tweet, Length: 46109, dtype: object

In [26]:
feature1

0          (وحاج,)
0           (حكي,)
0       (مثاليات,)
0          (وامه,)
0       (ومابعرف,)
           ...    
1169      (تكبير,)
1169        (نفخ,)
1169        (شفط,)
1169         (او,)
1169      (شفاف؟,)
Name: Tweet, Length: 11904, dtype: object

In [27]:
# Create a TfidfVectorizer object with the custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=arabic_tokenizer, stop_words=stopwords.words('arabic'))

# Fit and transform the 'Tweet' column in dfTrainN
tfidf_train = vectorizer.fit_transform(dfTrainN['Tweet'].apply(lambda x: ' '.join(x)))

# Print the shape of the transformed data
print("Shape of TF-IDF train data:", tfidf_train.shape)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


Shape of TF-IDF train data: (46109, 17461)


In [28]:
# Create a TfidfVectorizer object with the custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=arabic_tokenizer, stop_words=stopwords.words('arabic'))

# Fit and transform the 'Tweet' column in dfTrainN
tfidf_test = vectorizer.fit_transform(dfTestN['Tweet'].apply(lambda x: ' '.join(x)))

# Print the shape of the transformed data
print("Shape of TF-IDF train data:", tfidf_test.shape)

Shape of TF-IDF train data: (11904, 6428)


In [29]:
from sklearn.svm import SVC

# Create an SVM classifier
svm = SVC(kernel='linear')

# Fit the classifier to the training data
svm.fit(tfidf_test, target1)

# Transform the 'Tweet' column in dfTestN
tfidf_test = vectorizer.transform(dfTestN['Tweet'].apply(lambda x: ' '.join(x)))

# Predict the class labels for the test data
predictions = svm.predict(tfidf_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(dfTestN['Class_new'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8728158602150538


In [30]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(tfidf_train, dfTrainN['Class_new'], test_size=0.2, random_state=42)

# Initialize the SVM classifier
svm = SVC()

# Train the classifier on the training data
svm.fit(tfidf_test, target1)

# Evaluate the classifier on the test data
accuracy = svm.score(tfidf_test, target1)
print("Accuracy:", accuracy)

Accuracy: 0.8729838709677419


In [31]:
#pipe = make_pipeline(TfidfVectorizer(),LogisticRegression())

In [32]:
#param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

In [33]:
#model = GridSearchCV(pipe, param_grid, cv=2)
#model.fit(feature,target)